### NICO

Implementation based on the matlab code by M. Rabbat

In [ ]:
import numpy as np
from random import shuffle, random
from scipy.special import gamma as gamma_function
from scipy.special import comb
from scipy.sparse import csr_matrix
import itertools
from math import log

Initialize parameters

In [ ]:
n = 3 #number of nodes in the network
T = 100 #number of paths
Nm = 5 #number of nodes per path
# np.random.seed(2)
np.random.seed(1337)

In [ ]:
A = np.random.rand(n, n)
A = A / A.sum(axis = 1, keepdims=True)

In [ ]:
pi = np.random.rand(n, 1)
pi = pi / pi.sum(axis=0, keepdims=True)

Generate some paths according to this Markov model

In [ ]:
X = np.zeros((T, Nm))

In [ ]:
# Generate random numbers for testing purposes. Change to random() when it is ready
R_out = np.random.rand(T,1)
R_in = np.random.rand(T, Nm)

In [ ]:
cumprobs = pi.cumsum(axis = 0)

In [ ]:
iterator = 0
for walk in X:
#     Sample the starting node from Pi
#     larger = (cumprobs >= random()).nonzero()
    larger = (cumprobs >= R_out[iterator][0]).nonzero()
    walk[0] = larger[0][0]
#     Sample remaining nodes in the path by taking a random walk
    for i in range(1, Nm):
        cumprobs_in = A[int(walk[i - 1]),:].cumsum(axis=0)
#       larger = (cumprobs >= random()).nonzero()
        larger = (cumprobs_in >= R_in[iterator][i]).nonzero()
        walk[i] = larger[0][0]
    iterator += 1
X = X.astype(int)

Shuffle observations

In [ ]:
# Y = X.copy()
# for walk in Y:
#     shuffle(walk)

In [ ]:
# numTrials = 50

Utils

In [ ]:
def normalize_csr_rows(csr_mat):
    row_sums = np.array(csr_mat.sum(axis=1))[:,0]
    row_indices, col_indices = csr_mat.nonzero()
    csr_mat.data /= row_sums[row_indices]

In [ ]:
def normalize_csr_columns(csr_mat):
    col_sums = np.array(csr_mat.sum(axis=0))[0,:]
    row_indices, col_indices = csr_mat.nonzero()
    csr_mat.data /= col_sums[col_indices]

In [ ]:
def permutation_probabilities(bag_of_nodes, pi_hat, A_hat, full_probs = True):
    n = len(bag_of_nodes)
    
    permutation_orders = list(itertools.permutations(list(range(n))))
    
    gamma = np.zeros(n)
    Gamma = np.zeros((n,n))
    probability = 0
    
    for order in permutation_orders:
        starting_node = order[0]
        p = pi_hat[bag_of_nodes[starting_node]]
        
        for i in range(1, n):
            p = p * A_hat[bag_of_nodes[order[i - 1]], bag_of_nodes[order[i]]]
        
        if full_probs:
            gamma[order[0]] += p
        
            for i in range(1, n):
                Gamma[order[i - 1]][order[i]] += p
        else:
            probability += p
    
    if full_probs:
        return gamma, Gamma
    else:
        return probability

In [ ]:
def loglik(X, A, pi):
    ll = 0
    for walk in X:
        l = len(walk)
        p = 0
        for i in range(l):
            p = permutation_probabilities(walk, pi, A, full_probs=False)
        ll += log(p) - log(gamma_function(l + 1))
    return ll

In [ ]:
# def overlapped_chunks(array, chunk_size = 2, overlap_size = 1):
#     return [tuple(array[i:i+chunk_size]) for i in range(0, len(array)-1, chunk_size-overlap_size)]

NICO implementation

In [ ]:
# @TBD Change names. Come up with something meaningful and readable

def nico(X, n):
#     T = np.shape(X)[0]
    
    #number of nodes in each path
#     size = lambda array: len(array)
#     Nm = np.apply_along_axis(size, 1, X)
    
    #Init pi_hat
    #Assume all states appear at least once in the data
    pi_hat = 1 + 0.3 * np.random.rand(n, 1)
    pi_hat = pi_hat / pi_hat.sum(axis = 0, keepdims = True)
    pi_hat = [item[0] for item in pi_hat]
    
    # Construct A_hat as a sparse matrix
    # First determine an upperbound on the number of non-zero entries
    ii = []
    jj = []

    for walk in X:
        V = np.array(list(itertools.combinations(walk, 2)))
        ii.append(list(V[:, 0]))
        jj.append(list(V[:, 1]))

    ii = [item for sublist in ii for item in sublist]
    jj = [item for sublist in jj for item in sublist]
    ss = np.ones(len(ii))
    
    A_hat = csr_matrix((ss, (ii, jj)), shape = (n,n))
    A_hat = (A_hat + A_hat.transpose()) / 2
    A_hat_copy = A_hat.copy()
    A_hat_copy.data.fill(1)
    
    A_hat = A_hat_copy + 0.4 * csr_matrix((np.random.random((A_hat.nnz)),A_hat.nonzero()), shape=A_hat.shape)
    
    #Normalize A_hat
    normalize_csr_columns(A_hat)
    A_hat = A_hat.transpose()
    
    # EM algorithm
    tol = 0.01
    kmax = 100
    for k in range(kmax):
        # E-STEP
        #Test on one permutation
        r_alpha_gamma = []
        r_alpha_Gamma = []

        for bag_of_nodes in X:
            gamma, Gamma = permutation_probabilities(bag_of_nodes, pi_hat, A_hat)
            gamma_sum = sum(gamma)
            r_alpha_gamma.append(gamma/gamma_sum)
            r_alpha_Gamma.append(Gamma/gamma_sum)

        # M-STEP
        #1. Sum probabilities for gamma
        c = np.zeros(n)
        for seq, probs in zip(X, r_alpha_gamma):
            for node_id in range(n):
                node_indexes = [i for i, j in enumerate(seq) if j == node_id]
                c[node_id] += probs[node_indexes].sum()

        #2. Sum probabilities for Gamma
        C = np.zeros((n,n))
        for seq, probs in zip(X, r_alpha_Gamma):
            l = len(seq)
            for i in range(l - 1):
                for j in range(i + 1, l):
                    C[(seq[i],seq[j])] += probs[(i,j)]
                    C[(seq[j],seq[i])] += probs[(j,i)]

        A_hat_old = A_hat.copy()
        pi_hat_old = pi_hat.copy()

        A_hat = csr_matrix(C)
        pi_hat = c

        #Normalize
        pi_hat = pi_hat / pi_hat.sum(axis = 0, keepdims = True)
        normalize_csr_rows(A_hat)

        # Compute change in Q
        Q = 0
        Q_old = 0
        for seq, probs in zip(X, r_alpha_gamma):
            l = len(seq)
            for node_id in range(l):
                Q += probs[node_id] * log(pi_hat[seq[node_id]])
                Q_old += probs[node_id] * log(pi_hat_old[seq[node_id]])

        for seq, probs in zip(X, r_alpha_Gamma):
            l = len(seq)
            for i in range(l - 1):
                for j in range(i + 1, l):
                    Q += probs[(i,j)] * log(A_hat[(seq[i], seq[j])] + np.finfo(float).eps)
                    Q += probs[(j,i)] * log(A_hat[(seq[j], seq[i])] + np.finfo(float).eps)

                    Q_old += probs[(i,j)] * log(A_hat_old[(seq[i], seq[j])] + np.finfo(float).eps)
                    Q_old += probs[(j,i)] * log(A_hat_old[(seq[j], seq[i])] + np.finfo(float).eps)

        delta = Q - Q_old

        # Check stopping criterion
        sc = delta / tol
        
        print("Iter: {0} || Delta: {1:.2f} || Q: {2:.2f}".format(k + 1, delta, Q))
        
        if sc < 1:
            print("Terminated successfully after {0} iterations.".format(k + 1))
            break
            
        if k == kmax - 1:
            print("Number of EM iterations exceeded the limit.")
    
    return pi_hat, A_hat

In [ ]:
# %%time
# pi_hat, A_hat = nico(X, n)

NICO trials

In [ ]:
num_trials = 50
ll = []
l1 = []
for i in range(num_trials):
    print("Trial {0}:".format(i + 1))
    pi_hat, A_hat = nico(X, n)
    ll.append(loglik(X, A_hat, pi_hat))
    l1.append(np.sum(np.abs(A_hat - A)) + np.sum(np.abs(pi_hat - pi.flatten())))